# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/aqsa/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aqsa/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [7]:
!mkdir data

mkdir: data: File exists


In [8]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31493    0 31493    0     0  64831      0 --:--:-- --:--:-- --:--:-- 64934


In [9]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70519    0 70519    0     0   623k      0 --:--:-- --:--:-- --:--:--  626k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [10]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [11]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/aqsa/AI Projects/AIE6 Class Projects/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/aqsa/AI Projects/AIE6 Class Projects/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/aqsa/AI Projects/AIE6 Class Projects/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [16]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [17]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [18]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/20 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 11, relationships: 35)

We can save and load our knowledge graphs as follows.

In [19]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 11, relationships: 35)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [21]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [22]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Single Hop Specific Query: Goes to only one place to answer the query (Para says Albert Was born IN Jamaica. If the question is Where was Albert born? It answers using the single hop)

Multi Hop Abstract Query: Multipl sources, but not a simple conceptual question(not a fact). but it will take

Multi Hop Specific Query: Multiple sources/chunks, and answers a factual question.

0.25 (second paremeter) is the distribution

Finally, we can use our `TestSetGenerator` to generate our testset!

In [24]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Was 2023 a significant year for LLM development?,[My blog in 2023 Here’s the sequel to this pos...,"Yes, 2023 was a significant year for LLM devel...",single_hop_specifc_query_synthesizer
1,How does the term 'English' relate to the chal...,[openly licensed ones are still the world’s mo...,The context discusses the complexities of lang...,single_hop_specifc_query_synthesizer
2,Who is Simon Willison?,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison is mentioned in the context as ...,single_hop_specifc_query_synthesizer
3,What is the significance of the Gemini model i...,"[of what LLMs are, how they work and how they ...",The context mentions 'Image segmentation using...,single_hop_specifc_query_synthesizer
4,What significance does Princeton hold in the c...,[The rise of inference-scaling “reasoning” mod...,"In the context provided, Princeton is listed a...",single_hop_specifc_query_synthesizer
5,How does the reliance on vibes-based developme...,[<1-hop>\n\nopenly licensed ones are still the...,The context highlights that evaluating large l...,multi_hop_abstract_query_synthesizer
6,Considering the importance of careful training...,[<1-hop>\n\nof the best descriptions I’ve seen...,The context emphasizes that careful design of ...,multi_hop_abstract_query_synthesizer
7,how fine-tuning and community-driven model imp...,[<1-hop>\n\nThe rise of inference-scaling “rea...,The context shows that many models are being f...,multi_hop_abstract_query_synthesizer
8,How do Claude models compare to Gemini in term...,[<1-hop>\n\nThe rise of inference-scaling “rea...,"The context indicates that Claude models, such...",multi_hop_specific_query_synthesizer
9,How do the themes of Llama relate to the devel...,"[<1-hop>\n\nof what LLMs are, how they work an...",The context segments highlight the significanc...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [25]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/16 [00:00<?, ?it/s]

unable to apply transformation: 'StringIO' object has no attribute 'output'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [26]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is Mistral?,[My blog in 2023 Here’s the sequel to this pos...,Mistral is one of the organizations that has p...,single_hop_specifc_query_synthesizer
1,Large Language Models are what in AI?,[Simon Willison’s Weblog Subscribe Stuff we fi...,Large Language Models are the latest and most ...,single_hop_specifc_query_synthesizer
2,What is the significance of Gemini in the cont...,"[of what LLMs are, how they work and how they ...",The provided context mentions 'Image segmentat...,single_hop_specifc_query_synthesizer
3,What are the recent advancements and key theme...,[Things we learned about LLMs in 2024 31st Dec...,"In 2024, a lot has happened in the world of La...",single_hop_specifc_query_synthesizer
4,How do the high cost and resource requirements...,[<1-hop>\n\npunch massively above their weight...,The context explains that the cost of running ...,multi_hop_abstract_query_synthesizer
5,How do the capabilities and limitations of lar...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,Simon Willison’s weblog highlights that 2023 w...,multi_hop_abstract_query_synthesizer
6,how AI impact environment training inference a...,[<1-hop>\n\nThings we learned about LLMs in 20...,"The context shows that in 2024, the environmen...",multi_hop_abstract_query_synthesizer
7,how access to top models and their limitations...,[<1-hop>\n\nThings we learned about LLMs in 20...,"In 2024, the model rankings and leaderboard sh...",multi_hop_abstract_query_synthesizer
8,"Based on the insights from the blog posts, how...",[<1-hop>\n\nMy blog in 2023 Here’s the sequel ...,The blog posts highlight that recent advanceme...,multi_hop_specific_query_synthesizer
9,How does the development of GPT-4 relate to th...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2023, large language models like GPT-4 were...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [27]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [28]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [29]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [31]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [32]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

/Users/aqsa/AI Projects/AIE6 Class Projects/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/Users/aqsa/AI Projects/AIE6 Class Projects/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


In [33]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [34]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [35]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [36]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [37]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an infuriatingly vague term with no single, clear, and widely understood meaning. The two main interpretations are:\n\n1. AI agents as systems that go and act on your behalf, like a travel agent model.\n2. Large Language Models (LLMs) given access to tools which they can run in a loop to solve problems.\n\nOverall, agents are often described as AI systems that can operate autonomously on behalf of a user. However, despite much discussion and excitement, practical examples of agents running in production remain rare. The concept is closely tied to challenges such as gullibility of LLMs and may depend on achieving Artificial General Intelligence (AGI) to be truly effective.\n\nIn summary, "agents" are roughly AI systems intended to act independently for users, but the term is currently ambiguous and these systems are mostly still conceptual or experimental rather than fully realized.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [38]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [39]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

qa_eval - System level eval (unlike others )

## LangSmith Evaluation

In [40]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'timely-transport-74' at:
https://smith.langchain.com/o/4e223e9d-b789-4c00-8d16-32ad70974f10/datasets/13638fbc-6c17-4357-92ce-1c2c99ac6e9d/compare?selectedSessions=6f5ef352-3d61-4f01-bddb-1640abca8cbd




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,Whay are LLMs and how do they impact society a...,I don't know.,None,"LLMs, or Large Language Models, are a recent b...",0,0,0,2.514792,46daf30f-d1b6-43b6-9e22-dab389e9380d,cf793616-79a9-4329-907c-7ef2c2bb544d
1,how do LLMs work and what impact do they have ...,"Based on the provided context, LLMs (large lan...",None,"The context explains that LLMs, or large langu...",1,0,0,7.063276,a047e2af-6f65-47aa-99d1-a221f32fdf76,7c33638d-b6b9-4b3f-9823-fda75fb3f858
2,How does the development of GPT-4 relate to th...,The development of GPT-4 set a significant ben...,None,"In 2023, large language models like GPT-4 were...",1,1,0,5.630917,dbeef64d-6d16-4163-bac1-4f009533d29f,fe0fdc29-d846-4a79-ad35-d4885b7fb8fb
3,"Based on the insights from the blog posts, how...","Based on the provided context, recent developm...",None,The blog posts highlight that recent advanceme...,1,1,0,4.432277,dd8d30a7-e940-4ab6-8f74-2dbba44f7def,943d3924-1a57-4577-bdae-b33d245f16be
4,how access to top models and their limitations...,"Based on the provided context, access to top m...",None,"In 2024, the model rankings and leaderboard sh...",1,0,0,6.597011,9dfbb3da-0f15-4eb2-90fc-157fbaa87287,8b6ee7ef-bd47-4220-be02-d046219f5feb
5,how AI impact environment training inference a...,Based on the provided context:\n\n- The enviro...,None,"The context shows that in 2024, the environmen...",1,1,0,4.794420,048e08e7-30a8-4a97-8f00-1318855384f4,56ad502f-e76e-4b04-adfc-ccac9e484f1b
6,How do the capabilities and limitations of lar...,Based on Simon Willison’s weblog about AI in 2...,None,Simon Willison’s weblog highlights that 2023 w...,1,1,0,6.512255,7de9c4cb-cffc-4084-8c57-7d1d8d2b688c,2c059a0d-83d8-4198-ad47-46c7a38bf89b
7,How do the high cost and resource requirements...,The high cost and resource requirements for tr...,None,The context explains that the cost of running ...,1,1,0,2.896490,d8e91d14-0601-497b-9e0b-0774cc2e6d20,9bed8ddd-83b5-45a5-8f57-0c2e14a0d8f3
8,What are the recent advancements and key theme...,Recent advancements and key themes in Large La...,None,"In 2024, a lot has happened in the world of La...",1,1,0,5.557901,353223e0-0197-4ada-b356-4a09d984f61a,027671f2-7cca-4039-95e3-b853a1e436fc
9,What is the significance of Gemini in the cont...,"Gemini, particularly Google’s Gemini series, i...",None,The provided context mentions 'Image segmentat...,1,1,0,6.550157,49c030c3-6839-44b5-b677-d112663feba5,efd6240f-8f51-4b8f-8671-c740c0197699


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [41]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [42]:
rag_documents = docs

In [43]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [44]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [45]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [46]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [47]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [48]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Alright, here’s the lowdown on “Agents” straight from the context — Agents are kinda the elusive, hyped-up AI systems folks talk about that are supposed to act on your behalf, like a digital travel agent running errands for you. But here’s the kicker: the term “agents” is mad vague and all over the place. Some folks think of them as LLMs (large language models) with tool access running loops to solve problems, others imagine full-on autonomous AI doing your bidding.\n\nThe problem? These agents haven’t really shown up in the wild yet because of a big obstacle: gullibility. These AI systems are too trusting — they believe what you tell them, real or fake, which makes reliable decision-making tricky. We’re also stuck on issues like prompt injection and the inability to fact-check on the fly. So, while agents sound dope in theory, they still feel like “coming soon” because we’re not quite there yet. AGI might be the key to cracking this nut.\n\nSo, TL;DR: Agents = AI systems that act on 

Finally, we can evaluate the new chain on the same test set!

In [49]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'clear-produce-12' at:
https://smith.langchain.com/o/4e223e9d-b789-4c00-8d16-32ad70974f10/datasets/13638fbc-6c17-4357-92ce-1c2c99ac6e9d/compare?selectedSessions=f731796b-4d02-4a3d-aa09-e151b52cb603




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,Whay are LLMs and how do they impact society a...,"Aight, here’s the lowdown on LLMs—Large Langua...",None,"LLMs, or Large Language Models, are a recent b...",1,0,1,5.652788,46daf30f-d1b6-43b6-9e22-dab389e9380d,74a97ce1-5d0e-43f0-bc0e-c5b53e39c452
1,how do LLMs work and what impact do they have ...,"Yo, based on the vibe from the docs you droppe...",None,"The context explains that LLMs, or large langu...",1,0,1,9.066841,a047e2af-6f65-47aa-99d1-a221f32fdf76,c7dba547-6aba-42e1-a0cc-e76dcdd35c1d
2,How does the development of GPT-4 relate to th...,"Alright, check this out—GPT-4 was the OG spotl...",None,"In 2023, large language models like GPT-4 were...",1,1,1,4.959501,dbeef64d-6d16-4163-bac1-4f009533d29f,26595cfa-c65d-4050-909b-d388bf7d2ff6
3,"Based on the insights from the blog posts, how...","Alright, check this out—2023 was the year when...",None,The blog posts highlight that recent advanceme...,1,0,1,5.593862,dd8d30a7-e940-4ab6-8f74-2dbba44f7def,f60e33a3-2007-4db0-bd72-8194f86f8c1d
4,how access to top models and their limitations...,"Yo, here’s the lowdown on access and limitatio...",None,"In 2024, the model rankings and leaderboard sh...",1,1,1,5.198261,9dfbb3da-0f15-4eb2-90fc-157fbaa87287,687f065d-c3f7-467a-bbe8-0935bfcef565
5,how AI impact environment training inference a...,"Alright, listen up! When it comes to AI's envi...",None,"The context shows that in 2024, the environmen...",0,0,1,6.692405,048e08e7-30a8-4a97-8f00-1318855384f4,5c0fcf5b-8c06-4e1e-8ccb-520738439397
6,How do the capabilities and limitations of lar...,"Alright, here’s the lowdown straight from Simo...",None,Simon Willison’s weblog highlights that 2023 w...,1,1,1,5.854119,7de9c4cb-cffc-4084-8c57-7d1d8d2b688c,8af2f0d5-1a5b-43e0-8775-56e3e3f879e7
7,How do the high cost and resource requirements...,"Yo, check this out—training those massive lang...",None,The context explains that the cost of running ...,1,0,1,4.328648,d8e91d14-0601-497b-9e0b-0774cc2e6d20,294bf638-5ea4-4d14-af27-b5b6ed77ea03
8,What are the recent advancements and key theme...,"Yo, here’s the lowdown straight from the 2024 ...",None,"In 2024, a lot has happened in the world of La...",1,0,1,6.778459,353223e0-0197-4ada-b356-4a09d984f61a,eac2ae77-9f79-43fc-a76e-400b2583800c
9,What is the significance of Gemini in the cont...,"Alright, here’s the lowdown on Gemini in the L...",None,The provided context mentions 'Image segmentat...,1,1,1,9.030623,49c030c3-6839-44b5-b677-d112663feba5,9cd731dc-4728-422a-b24c-370dce1788e1


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

![label](Assignment.png)
![description]('documents/Assignment.png')

